# 性別判定AI開発(カラー画像)


# 画像データ収集

In [ ]:
from icrawler.builtin import BingImageCrawler
# 収集対象（検索キーワード）を指定する
keywords = {"男":"男性","女":"女性"}
for dir_name, keyword in keywords.items():    
    # 画像収集インスタンスの生成
    icrawler = BingImageCrawler(storage={"root_dir":"./data/human/"+dir_name})
    # 収集開始
    icrawler.crawl(keyword=keyword, max_num=100)

# テストデータ収集


In [ ]:
from icrawler.builtin import BingImageCrawler
# 収集対象（検索キーワード）を指定する
keywords = {"man":"男性","woman":"女性"}
for dir_name, keyword in keywords.items():    
    # 画像収集インスタンスの生成
    icrawler = BingImageCrawler(storage={"root_dir":"./data/test/"+dir_name})
    # 収集開始
    icrawler.crawl(keyword=keyword, max_num=5)


# 1 学習として不適切な画像を除去

humanフォルダから性別違い、横向き、肌の色が特殊な画像など、学習させるのに不向きな画像を除去する

※除去したのち男フォルダと女フォルダの画像枚数をそろえる

# 2 学習・検証用データ作成

In [ ]:
import glob
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pickle
out_dir = "./data/human/"
im_size = 50
im_color=3
save_file = out_dir + "/humandata2.pickle"
#人間の画像が入っているディレクトリから画像を取得
humandir = ["man","woman"]
result = []
for i, dir in enumerate(humandir):
    img_dir = out_dir + dir
    fs = glob.glob(img_dir+"/*")
    print("dir="+img_dir)
    for j, f in enumerate(fs):
        img = cv2.imread(f)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (im_size, im_size))
        result.append([i, img])
        if j == 5:
            plt.subplot(2, 2, i+1)
            plt.axis("off")
            plt.title(str(i))
            plt.imshow(img)
# ラベルと画像データを保存
pickle.dump(result, open(save_file, "wb"))
plt.show()

# 3 データ取得と変換

In [ ]:
# 初期処理
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pickle
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt

# データファイルと画像サイズの指定
data_file = "./data/human/humandata2.pickle"
im_size = 50
im_color = 3
in_size = im_size**2*im_color
out_size = 2 # 0:男 1:女


In [ ]:
data = pickle.load(open(data_file, 'rb'))

# 画像データを0-1の範囲に直す
y=[]
x=[]

for d in data:
    (num, img) = d
    img = img.reshape(-1).astype('float32')/255
    y.append(keras.utils.to_categorical(num, out_size))
    x.append(img)
x = np.array(x)
y = np.array(y)

# 3 データの分割

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

In [ ]:
x_train.shape

# 4 モデル構造の定義・モデルの構築

In [ ]:
model = Sequential()
model.add(Dense(512, activation = "relu", input_shape =(in_size,)))
model.add(Dense(out_size, activation = "softmax"))
model.summary()

# 5 学習の実行・モデルの評価

In [ ]:
# 学習を実行
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'],
)

In [ ]:
hist=model.fit(
        x_train, y_train,
          batch_size=20, # batch_sizeは、1回の学習で利用するデータ。1回のepochsで全データ数/batch_size分学習する。
          epochs=100,
          verbose=1,
          validation_data=(x_test, y_test)) # 学習回数

In [ ]:
# モデルを評価
score = model.evaluate(x_test, y_test, verbose=1)
print('正解率=', score[1], 'loss=', score[0])

In [ ]:
#正解率の推移をプロット
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Accuracy')
plt.legend(['train','test'], loc='upper left')

In [ ]:
#lossの推移をプロット
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Loss')
plt.legend(['train','test'], loc='upper left')

In [ ]:
# モデルの保存
model.save('./data/human_model2.h5')

# 学習済み重みデータを保存
model.save_weights('./data/human_weight2.h5')

# データを予測

- 0：男
- 1：女

In [ ]:
# 用意した画像を判定
import cv2
import matplotlib.pyplot as plt
from keras.models import load_model
im_size=50
label = {0:"男", 1:"女"}
# データの予測関数
model = load_model("./data/human_model2.h5")
model.load_weights('./data/human_weight2.h5')
def predict(filename):
    # 用意した画像ファイルを読み込む
    my_img = cv2.imread(filename)
    #読みこんだ画像ファイルを学習時の画像サイズに合わせる
    my_img = cv2.resize(my_img,(im_size, im_size))
    my_img = cv2.cvtColor(my_img, cv2.COLOR_RGB2BGR)
    plt.imshow(my_img,)
    plt.show()
    my_img = my_img.reshape(1,7500).astype('float32') 
    
    # データを予測する
    result = model.predict(my_img)
    dic= label[np.argmax(result[0])]
    per = result[0][np.argmax(result[0])]
    return dic,per
# 画像データを指定して予測関数を実行
testfile = './data/test/woman/000002.jpg' # 女
dic,per = predict(testfile)
print(testfile , '=', dic,per)

In [ ]:
# 画像データを指定して予測関数を実行
testfile = './data/test/man/000003.jpg' # 男
dic,per = predict(testfile)
print(testfile , '=', dic,per)